# Import

In [9]:
import csv
import os
import re
import sys

import numpy as np
import pandas as pd
from nltk import download, word_tokenize
from nltk.corpus import stopwords
from stop_words import get_stop_words
from tqdm import tqdm

from urlextract import URLExtract

## Add configuration file

In [10]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [11]:
from ALL import config 
from util import *

## Set condition

In [12]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [13]:
s3 = S3Manager()

# Read data

In [14]:
df_path = s3.download("DataShaping/TweetTopic/master.csv")

In [15]:
df = pd.read_csv(df_path[0], index_col=0)

In [16]:
df = df.reset_index(drop=True)

In [17]:
labels_path = s3.download("DataShaping/TweetTopic/class.csv")

In [18]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [19]:
download('punkt')
download("stopwords")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## delete URL

In [20]:
def format_tweet(tweet):
    # mask web urls
    urls = extractor.find_urls(tweet)
    for url in urls:
        tweet = tweet.replace(url, "")
    # format twitter account
    tweet = re.sub(r"\b(\s*)(@[\S]+)\b", r'\1{\2@}', tweet)
    return tweet

In [21]:
extractor = URLExtract()
df["text"] = df.text.progress_apply(format_tweet)

 25%|██▍       | 1718/6997 [00:02<00:07, 742.78it/s]

In [22]:
df["words"] = df.text.progress_apply(word_tokenize)

In [23]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve","'re", "'d", "'ll", "", "-+", "+-", "_/", "||", "__", "/|", "//", "{", "}"]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [24]:
#     一文字以下の単語とstop_word, stop_charを削除
df["words_nonstop"] = df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word)> 1
    ]
)

In [25]:
df.words = df.words.progress_apply(lambda words: " ".join(words))
df.words_nonstop = df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

# output

## make file

In [26]:
with open(make_filepath(f"../../temporary/Preprocessing/TweetTopic/class.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(class_labels)
df.to_csv(make_filepath(f"../../temporary/Preprocessing/TweetTopic/master.csv"))

## upload file

In [27]:
s3.upload("../../temporary/Preprocessing/TweetTopic", "Preprocessing/TweetTopic/")

In [28]:
s3.delete_local_all()

In [29]:
shutil.rmtree("../../nltk_data/")